# Notebook Contents and Objective:
This notebook contains details experiment of model evaluation and inference part

In [ ]:
!pip install -q transformers
!pip install -q datasets
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import os
import json
from google.colab import data_table
import datasets
from datasets import Dataset
import json
# from thefuzz import process
# from thefuzz import fuzz
import re
import random
from typing import Any, List, Tuple

from PIL import Image
import torch
print("CUDA available:", torch.cuda.is_available())
# from donut import DonutModel
import transformers
from transformers import DonutProcessor, VisionEncoderDecoderConfig, VisionEncoderDecoderModel, BartConfig
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from torch.optim import Adam
from tqdm.auto import tqdm
from torch.nn.utils.rnn import pad_sequence
from nltk import edit_distance

CUDA available: False


## S-1: Initial Steps of Data Preparation
- We can skip these steps till gt_parse or ground truth and we can take the processed dataset from `https://huggingface.co/indra-ami`

In [ ]:
## Custom defined function for processing the data
def process_data(main_json_path, base_dir_path):

  # Open the JSON file
  with open(main_json_path, 'r') as file:
      # Load the contents of the file into a dictionary
      data = json.load(file)

  # Convert 'data' list into a dataframe
  df = pd.DataFrame(data['data'])

  df = df.iloc[:9]
  # Define a function to join the base path with the image path
  def join_paths(image_path):
      return os.path.join(base_dir_path, image_path)

  # Apply the function to create a new column with the full image path
  df['full_path_image'] = df['image'].apply(join_paths)

  def extract_ocr_info(row, ocr_base_dir):

    # Construct the ocr_path for the row
    ocr_path = os.path.join(ocr_base_dir, row['ucsf_document_id'] + '_' + row['ucsf_document_page_no'] + '.json')

    # Open the json file and load the data
    with open(ocr_path, 'r') as f:
      ocr_json = json.load(f)

    # Extract the bounding boxes and word lists from the json data
    bbox_row = []
    word_row = []
    for i in ocr_json['recognitionResults'][0]['lines']:
      for j in i['words']:
        bbox_row.append(j['boundingBox'])
        word_row.append(j['text'])

    # Return the bounding boxes and word lists
    return bbox_row, word_row

  ocr_base_dir = os.path.join(base_dir_path, os.listdir(base_dir_path)[1])

  # Apply the extract_ocr_info function to each row in the DataFrame
  df['bounding_boxes'], df['word_list'] = zip(*df.apply(lambda row: extract_ocr_info(row, ocr_base_dir), axis=1))

  df.drop(['questionId', 'image', 'ucsf_document_id', 'ucsf_document_page_no'], axis=1, inplace=True)

  return df



In [ ]:
## Processing of all three Train, Validation and Test dataset

# training usage:
train_path_main = '/content/drive/MyDrive/docVqa_dataset/train/train_v1.0.json'
base_train_path = '/content/drive/MyDrive/docVqa_dataset/train'

df_train_sub_ocr = process_data(train_path_main, base_train_path)
# Convert dataframe into dataset object
hf_train_sub_ocr = Dataset.from_pandas(df_train_sub_ocr)

# validation usage:
valid_path_main = '/content/drive/MyDrive/docVqa_dataset/val/val_v1.0.json'
base_valid_path = '/content/drive/MyDrive/docVqa_dataset/val'

df_valid_sub_ocr = process_data(valid_path_main, base_valid_path)
# Convert dataframe into dataset object
hf_valid_sub_ocr = Dataset.from_pandas(df_valid_sub_ocr)

# test usage:
test_path_main = '/content/drive/MyDrive/docVqa_dataset/test/test_v1.0.json'
base_test_path = '/content/drive/MyDrive/docVqa_dataset/test'

df_test_sub_ocr = process_data(test_path_main, base_test_path)
# Convert dataframe into dataset object
hf_test_sub_ocr = Dataset.from_pandas(df_test_sub_ocr)

In [ ]:
# Creating custom function for DOCVQA task
def add_ground_truth(hf_examples):
  images = hf_examples['full_path_image']
  questions = hf_examples['question']
  answers = hf_examples['answers']

  ground_truths = []    # Creating empty list
  for image,question, answer in zip(images,questions, answers):
    # we need to escape " characters appearing in the query and/or answer
    question = question.replace("\\", "") ## this was just one corrupt example (index 91 of training set)
    question = re.sub(' +', ' ', question)
    question = question.replace('"', '\\"')   # replacement of " from question
    # let's create the ground truth string
    ground_truth_example = '{"gt_parses": ['
    for idx, answ in enumerate(answer):
      # ans = answ.replace('"', '\"')   # replacement of " from answer
      answ = answ.replace("\\", "")
      ans = answ.replace('"', '\\"')
      ground_truth_example += '{"question" : "' + question + '", "answer" : "' + ans + '"}'

      # add comma for more than one element present in the answer list
      if idx != len(answer) - 1:   # when current index is not equal to last index
        ground_truth_example += ', '
    ground_truth_example += ']}'
    ground_truths.append(ground_truth_example)    ## appending ground_truths list for every row

  hf_examples['ground_truth'] = ground_truths

  return hf_examples

In [ ]:
## Creating ground truth string for all three dataset train, valid and test dataset

# For Training
hf_train_sub_ocr_gt = hf_train_sub_ocr.map(add_ground_truth, batched=True)

# For Validation
hf_valid_sub_ocr_gt = hf_valid_sub_ocr.map(add_ground_truth, batched=True)

# *** Note : For Test data as there is no such answers column so we will not processed further
# hf_test_sub_ocr_gt = hf_test_sub_ocr.map(add_ground_truth, batched=True)

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

In [ ]:
hf_train_sub_ocr_gt

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list', 'ground_truth'],
    num_rows: 9
})

In [ ]:
hf_valid_sub_ocr_gt

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list', 'ground_truth'],
    num_rows: 9
})

In [ ]:
# model_checkpoint = "naver-clova-ix/donut-base"
model_checkpoint = "naver-clova-ix/donut-base-finetuned-docvqa"
max_length = 128
image_size = [1280, 960]


config = VisionEncoderDecoderConfig.from_pretrained(model_checkpoint)
config.encoder.image_size = image_size # (height, width)
# update max_length of the decoder (for generation)
config.decoder.max_length = max_length

processor = DonutProcessor.from_pretrained(model_checkpoint)
model = VisionEncoderDecoderModel.from_pretrained(model_checkpoint, config=config)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [ ]:
hf_train_sub_ocr_gt

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list', 'ground_truth'],
    num_rows: 9
})

In [ ]:
hf_train_sub_ocr_gt['ground_truth'][1], type(hf_train_sub_ocr_gt['ground_truth'][1])

('{"gt_parses": [{"question" : "what is the contact person name mentioned in letter?", "answer" : "P. Carter"}, {"question" : "what is the contact person name mentioned in letter?", "answer" : "p. carter"}]}',
 str)

In [ ]:
len(hf_train_sub_ocr_gt)

9

In [ ]:
type(hf_train_sub_ocr_gt)

datasets.arrow_dataset.Dataset

In [ ]:
# Image.open(hf_train_sub_ocr_gt['full_path_image'][1]).convert("RGB")

In [ ]:
processor(Image.open(hf_train_sub_ocr_gt['full_path_image'][1]).convert("RGB")).pixel_values

[array([[[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ...,
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.]],
 
        [[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ...,
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.]],
 
        [[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ...,
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.]]], dtype=float32)]

## S-2: Creating PyTorch Donut Dataset Class

In [ ]:
from torch.utils.data import Dataset

In [ ]:
## Final Class by combining altogether

new_special_tokens_list = []
# list with the newly added tokens. The purpose of this list is to keep track of the tokens that were added to the tokenizer.

## DonutDataset class inherits from the torch.utils.data.Dataset class and overrides its methods

class DonutDataset(Dataset):
  """
  Args:
      dataset_name_or_path: name of dataset (available at huggingface.co/datasets) or the path containing image files and metadata.jsonl
      max_length: the max number of tokens for the target sequences
      split: whether to load "train", "validation" or "test" split
      ignore_id: ignore_index for torch.nn.CrossEntropyLoss
      prompt_start_token: the special token at the start of the sequences
      prompt_end_token: the special token at the end of the sequences
      sort_json_key: whether or not to sort the JSON keys
  """

  def __init__(
    self,
    dataset_name_or_path: str,    # Name of the dataset or the path containing huggingface datset
    max_length: int,              # the maximum number of tokens for the target sequences/prompt
    split: str = None,
    ignore_id: int = -100,        # ignore_index for torch.nn.CrossEntropyLoss
    prompt_start_token: str = None,   # The special token at the start of the propmpt sequences
    prompt_end_token: str = None,   # The special token at the end of the prompt sequences
    sort_json_key: bool = True,       # Whether or not to sort the JSON keys
):
    super().__init__()

    ## Variable Declaration

    self.max_length = max_length
    self.split = split
    self.ignore_id = ignore_id
    self.prompt_start_token = prompt_start_token
    self.prompt_end_token = prompt_end_token
    self.sort_json_key = sort_json_key

    self.dataset = dataset_name_or_path
    self.dataset_length = len(self.dataset)   # The length of the dataset is stored

    self.gt_prompt_sequences = []    # operation for every single row  *** Note: same as hf_examples['gt_prompt'] which we did for single custom function case
    # list stores the ground truth (gt) token sequences, which are generated using the json2token() method for each sample in the dataset

    ## S-1: Iterate over all rows of the dataset and creating gt_prompt from gt_parses

    for sample in self.dataset:   # each sample is a dictionary

      # print(sample["ground_truth"], '|||||', type(sample["ground_truth"]))

      ground_truth_jsob_obj = json.loads(sample["ground_truth"])   # convert a string representation of ground_truth a JSON object
      if "gt_parses" in ground_truth_jsob_obj:  # when multiple ground truths are available, e.g., docvqa
        assert isinstance(ground_truth_jsob_obj["gt_parses"], list)
        gt_jsons_list = ground_truth_jsob_obj["gt_parses"]    ## Stores the list of gt_parses value as gt_parses is the only key
      else:
        assert "gt_parse" in ground_truth_jsob_obj and isinstance(ground_truth_jsob_obj["gt_parse"], dict)
        gt_jsons_list = [ground_truth_jsob_obj["gt_parse"]]


      ## Appending the prompt using list comprehension with adding cls_token before and eos_token at the end
      # For DOCVQA task
      self.gt_prompt_sequences.append(
            [
              processor.tokenizer.cls_token +
              self.json2token(
                  # gt_jsons_list,
                  gt_json,
                  # update_special_tokens_for_json_key=self.split == "train",
                  sort_json_key=self.sort_json_key,
                )
              + processor.tokenizer.eos_token

              for gt_json in gt_jsons_list  # iterate over every dictionary json from the gt_jsons list
            ]
        )

    # self.add_tokens([self.prompt_start_token, self.prompt_end_token])
    self.add_tokens([self.prompt_start_token])
    self.prompt_start_token_id = processor.tokenizer.convert_tokens_to_ids(self.prompt_start_token)
    # self.prompt_end_token_id = processor.tokenizer.convert_tokens_to_ids(self.prompt_end_token)

    # print(self.gt_prompt_sequences)

  ## S-2: Function converts an ordered JSON object into a token sequence
  def json2token(self, json_obj: Any,  sort_json_key: bool = True):
    """
    Convert an ordered JSON object into a token sequence
    """
    if type(json_obj) == dict:
      if len(json_obj) == 1 and "text_sequence" in json_obj:
        return json_obj["text_sequence"]
      else:
        output = ""
        if sort_json_key:
          keys = sorted(json_obj.keys(), reverse=True)
        else:
          keys = json_obj.keys()

        for k in keys:
          # if update_special_tokens_for_json_key:
            # self.add_tokens([fr"", fr""])
          self.add_tokens([fr"<s_{k}>", fr"</s_{k}>"])
          output += (
              fr"<s_{k}>"
              + self.json2token(json_obj[k], sort_json_key)
              + fr"</s_{k}>"
          )
        return output

    elif type(json_obj) == list:
      return r"<sep/>".join([self.json2token(item, sort_json_key) for item in json_obj])

    else:
      json_obj = str(json_obj)
      if f"<{json_obj}/>" in new_special_tokens_list:
        json_obj = f"<{json_obj}/>"  # for categorical special tokens
      return json_obj

  ## S-3: Custom function to add new starting and ending prompt tokens and embed it
  def add_tokens(self, list_of_tokens: List[str]):
    """
    Add special tokens to tokenizer and resize the token embeddings of the decoder
    """
    newly_added_num = processor.tokenizer.add_tokens(list_of_tokens)
    if newly_added_num > 0:
      model.decoder.resize_token_embeddings(len(processor.tokenizer))
      new_special_tokens_list.extend(list_of_tokens)

  ## S-4: Aligns the dataset length with the valid indices
  def __len__(self) -> int:
    return self.dataset_length - 1

  ## S-5: special method that allows objects to support indexing and accessing their elements using square bracket notation ([]).
  # It is invoked when you use square brackets to access an element of an object, like obj[key]
  def __getitem__(self, indx: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Load image from image_path of given dataset_path and convert into input pixel value tensor
    Convert ground truth data into input_ids (tokenized string) which is nothing but decoder input ids
    Create clone tensor of decoder input ids for masking and model understanding task
    Returns:
        input_tensor : preprocessed image
        input_ids : tokenized gt_data
        labels : masked labels (model doesn't need to predict prompt and pad token)
    """
    sample = self.dataset[indx]

    # S-5.1: create input_tensor from image i.e pixel values
    try:
      ## ** Note: This image path part will vary as per task
      # pixel_values = processor(Image.open(sample['full_path_image']).convert("RGB"), random_padding=self.split == "train", return_tensors="pt").pixel_values
      pixel_values = processor(Image.open(sample['full_path_image']).convert("RGB"), return_tensors="pt").pixel_values
      # torch.Size([1, 3, 2560, 1920]) shape before squeeze
      input_tensor_pixel = pixel_values.squeeze()

    except Exception as e:
      print(sample)
      print(f"Error: {e}")
      return {}

    # S-5.2: tokenize document and creating decoder input ids

    # processed_prompt = self.gt_prompt_sequences[indx]  # can be more than one, e.g., DocVQA Task 1
    # random.choice() is in scenarios where you need to select a random element from a collection of items.
    processed_gt_prompt = random.choice(self.gt_prompt_sequences[indx])  # can be more than one, e.g., DocVQA Task 1
    decoder_input_ids = processor.tokenizer(
        processed_gt_prompt,
        add_special_tokens=False,
        max_length=self.max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"].squeeze(0)   #torch.Size([1, 128]) before squeeze

    # S-5.3: Create labels which is clone tensor for extracted values from decoder input ids tensors

    if self.split == "train":

      labels = decoder_input_ids.clone()    ## clone tensor of decoder input ids

      prompt_start_token_id_index = (labels == self.prompt_start_token_id).nonzero
      # prompt_end_token_id_index = (labels == self.prompt_end_token_id).nonzero

      labels[
          labels == processor.tokenizer.pad_token_id
      ] = self.ignore_id

      labels[
          : prompt_start_token_id_index().squeeze() + 1
      ] = self.ignore_id

      # ## Creating empty integer unmask tensor
      # unmask = torch.tensor([], dtype=torch.int)

      # ## iterate over both start and end tensor simulteneously
      # for start_index,end_index in zip(prompt_start_token_id_index(), prompt_end_token_id_index()):
      #   val_extracted = labels[start_index.squeeze()+1:end_index.squeeze()]     ## stores all extarcted values
      #   unmask = torch.cat((unmask, val_extracted))     ## concatenating start and end index value

      # labels[~torch.isin(labels, unmask)] =  self.ignore_id       ## assign with ignore id

      return input_tensor_pixel, decoder_input_ids, labels

    else:

      prompt_start_index = ((decoder_input_ids == self.prompt_start_token_id).nonzero)().squeeze()  # return prompt end index instead of target output labels

      return input_tensor_pixel, decoder_input_ids, prompt_start_index, processed_gt_prompt




In [ ]:
processor.image_processor.size = image_size[::-1] # should be (width, height)
processor.image_processor.do_align_long_axis = False

train_dataset_sub = DonutDataset(hf_train_sub_ocr_gt, max_length=max_length,split="train",
                            prompt_start_token="<s_answer>",
                            prompt_end_token="</s_answer>",
                            sort_json_key=False,
                             )

valid_dataset_sub = DonutDataset(hf_valid_sub_ocr_gt, max_length=max_length,split="valid",
                            prompt_start_token="<s_answer>",
                            prompt_end_token="</s_answer>",
                            sort_json_key=False,
                             )

In [ ]:
train_dataset_sub, len(train_dataset_sub)

(<__main__.DonutDataset at 0x7ff7782b6e00>, 8)

In [ ]:
valid_dataset_sub, len(valid_dataset_sub)

(<__main__.DonutDataset at 0x7ff7782b7d30>, 8)

In [ ]:
train_dataset_sub[0]

(tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]),
 tensor([    0, 57529, 27973, 52743, 48941, 44960, 40053, 35647, 25515, 54964,
   

In [ ]:
## Sample check of row 0 of train dataset
pixel_values_0, decoder_input_ids_0, labels_0  = train_dataset_sub[0]
for decoder_input_id, label in zip(decoder_input_ids_0.tolist()[:-1], labels_0.tolist()[1:]):
  if label != -100:
    print(processor.decode([decoder_input_id]), processor.decode([label]))
  else:
    print(processor.decode([decoder_input_id]), label)

<s> -100
<s_question> -100
what -100
is -100
the -100
date -100
mention -100
ed -100
in -100
this -100
letter -100
? -100
</s_question> -100
<s_answer> 1
1 /8
/8 /
/ 93
93 </s_answer>
</s_answer> </s>
</s> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -1

In [ ]:
## Sample check of row 1 from training dataset
pixel_values_1, decoder_input_ids_1, labels_1  = train_dataset_sub[1]
for decoder_input_id, label in zip(decoder_input_ids_1.tolist()[:-1], labels_1.tolist()[1:]):
  if label != -100:
    print(processor.decode([decoder_input_id]), processor.decode([label]))
  else:
    print(processor.decode([decoder_input_id]), label)

<s> -100
<s_question> -100
what -100
is -100
the -100
contact -100
person -100
name -100
mention -100
ed -100
in -100
letter -100
? -100
</s_question> -100
<s_answer> P
P .
. Car
Car ter
ter </s_answer>
</s_answer> </s>
</s> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100


In [ ]:
valid_dataset_sub[0]

(tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]),
 tensor([    0, 57529, 42543, 52743, 48941, 22621, 26468, 39288, 33855, 37448,
   

In [ ]:
## Sample check of row 1 from valid dataset
pixel_values_1, decoder_input_ids_1, prompt_start_index_1, processed_gt_prompt_1  = valid_dataset_sub[1]
print(pixel_values_1)
print('******************************************')
print(processor.decode(decoder_input_ids_1))
print('******************************************')
print(prompt_start_index_1)
print('******************************************')
print(processed_gt_prompt_1)

tensor([[[ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         ...,
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]],

        [[ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         ...,
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]],

        [[ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         ...,
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]]])
******************************************
<s><s_question> What is name of university?</s_question><s_answ

## S-3: Model Training Phase

### Model Training when Batch Size = 2
- Note:
len(train_dataloader) and len(train_dataset) will have the same value. This is because len(train_dataloader) represents the number of batches in the dataloader, and when the batch size is 1, each sequence is treated as a separate batch.

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataloader_sub = DataLoader(train_dataset_sub, batch_size=2)
valid_dataloader_sub = DataLoader(valid_dataset_sub, batch_size=2)

In [ ]:
len(train_dataloader_sub), len(valid_dataloader_sub)

(4, 4)

In [ ]:
batch = next(iter(train_dataloader_sub))
pixel_values, decoder_input_ids, labels = batch
print(pixel_values.shape)
print(decoder_input_ids.shape)
print(labels.shape)

torch.Size([2, 3, 1280, 960])
torch.Size([2, 128])
torch.Size([2, 128])


In [ ]:
pixel_values

tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1.

In [ ]:
decoder_input_ids

tensor([[    0, 57529, 27973, 52743, 48941, 44960, 40053, 35647, 25515, 54964,
         36322, 36209, 57524, 57526,  1314, 27810, 47106,  4050, 57523,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,  

In [ ]:
labels

tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  1314, 27810, 47106,  4050, 57523,     2,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  

In [ ]:
for decoder_input_id, label in zip(decoder_input_ids[0].tolist()[:-1][:30], labels[0].tolist()[1:][:30]):
  if label != -100:
    print(processor.decode([decoder_input_id]), processor.decode([label]))
  else:
    print(processor.decode([decoder_input_id]), label)

<s> -100
<s_question> -100
what -100
is -100
the -100
date -100
mention -100
ed -100
in -100
this -100
letter -100
? -100
</s_question> -100
<s_answer> 1
1 /8
/8 /
/ 93
93 </s_answer>
</s_answer> </s>
</s> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100


In [ ]:
len(processor.tokenizer)

57532

In [ ]:
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(["<s_answer>"])[0]
## Note:
# Setting the decoder start token ID in the model's configuration to the ID of the <s_answer> token in the tokenizer.
# This token is used to indicate the start of the target sequence during decoding.

In [ ]:
# list(model.parameters())

In [ ]:
## List of named parameter
# model.state_dict()

In [ ]:
## Finding Training Accuracy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model = model.to(device)


# Set the optimizer
optimizer = Adam(model.parameters(), lr=0.0001)

# Set the number of training epochs
num_epochs = 5

for epoch in range(num_epochs):

  # Training loop
  model.train()

  print(f"Epoch {epoch + 1}/{num_epochs}")

  total_loss = 0.0    # Total loss across all batches in an epoch
  total_correct = 0   # Total correct across all batches in an epoch
  total_samples = 0   # Effective when batch_size is more than 1
  total_samples_mask_level = 0 # Total number of non-padding and ignored tokens

  # Iterate over the training dataset
  for pixel_values, decoder_input_ids, labels in tqdm(train_dataloader_sub):

    pixel_values = pixel_values.to(device)
    decoder_input_ids = decoder_input_ids.to(device)
    labels = labels.to(device)

    ## Finding outputs prediction
    outputs = model(pixel_values=pixel_values, decoder_input_ids=decoder_input_ids[:, :-1], labels=labels[:, 1:])
    # print(outputs['loss'].shape, outputs['logits'].shape, outputs['encoder_last_hidden_state'].shape)

    ## Compute  the Loss
    loss = outputs.loss
    total_loss += loss.item()

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    # Compute the Accuracy (3-Ways)
    ## Pro-2
    total_samples += labels.size(0)   # # labels.size(0) = 1

    ## Pro-3 Mask level
    # Create a mask to exclude padding tokens & ignored tokens from accuracy calculation
    mask = (labels != -100)
    # Update the total number of non-padding tokens
    total_samples_mask_level += mask.sum().item()

    # Finding logits value
    logits = outputs.logits
    predictions = logits.argmax(dim=-1) # index value of maximum score for each token
    correct = (predictions == labels[:, 1:]).sum().item()    ## Number of correct in each row
    total_correct += correct    # Cumulative correct then lastly total correct prediction from each epoch

  ## Average Loss Calculation
  avg_epoch_loss_batch_level = total_loss / len(train_dataloader_sub)   # average loss per batch i.e. normalize the loss value

  ## Average Accuracy Calculation
  avg_epoch_accuracy_batch_level = total_correct / len(train_dataloader_sub) # average per epoch accuracy at batch level (meaningful when batch size > 1)

  avg_epoch_accuracy_sample_level = total_correct / total_samples # average per epoch accuracy at based total number of data point

  avg_epoch_accuracy_mask_level = (total_correct / total_samples_mask_level) * 100 # average per epoch accuracy at based on mask or non padding tokens

  # If batch_size = 1 , so we can use anything len(train_dataloader_sub) or  len(train_dataset)

  print(f"Training Loss Batch Level: {avg_epoch_loss_batch_level:.4f} :::: Training Accuracy Batch Level: {avg_epoch_accuracy_batch_level:.3f}%")
  print(f"Training Accuracy Entire Sample Level: {avg_epoch_accuracy_sample_level:.3f}% :::: Training Accuracy Mask Level: {avg_epoch_accuracy_mask_level:.3f}%")
  print('*****************************************************************************************************************************')




Epoch 1/5


  0%|          | 0/4 [00:00<?, ?it/s]

Training Loss Batch Level: 3.8231 :::: Training Accuracy Batch Level: 9.500%
Training Accuracy Entire Sample Level: 4.750% :::: Training Accuracy Mask Level: 63.333%
*****************************************************************************************************************************
Epoch 2/5


  0%|          | 0/4 [00:00<?, ?it/s]

Training Loss Batch Level: 2.0260 :::: Training Accuracy Batch Level: 10.750%
Training Accuracy Entire Sample Level: 5.375% :::: Training Accuracy Mask Level: 71.667%
*****************************************************************************************************************************
Epoch 3/5


...

Training Loss Batch Level: 1.1391 :::: Training Accuracy Batch Level: 12.250%
Training Accuracy Entire Sample Level: 6.125% :::: Training Accuracy Mask Level: 81.667%
*****************************************************************************************************************************
Epoch 4/5


  0%|          | 0/4 [00:00<?, ?it/s]

Training Loss Batch Level: 0.4680 :::: Training Accuracy Batch Level: 13.500%
Training Accuracy Entire Sample Level: 6.750% :::: Training Accuracy Mask Level: 90.000%
*****************************************************************************************************************************
Epoch 5/5


  0%|          | 0/4 [00:00<?, ?it/s]

Training Loss Batch Level: 0.2076 :::: Training Accuracy Batch Level: 14.250%
Training Accuracy Entire Sample Level: 7.125% :::: Training Accuracy Mask Level: 95.000%
*****************************************************************************************************************************


## S-4: PyTorch Model Evaluation Phase
- We will do the model evaluation using both edit distance and cosine similarity

In [ ]:
config = {
    "verbose": True
}

In [ ]:
## Validation section using edit distance
model.eval()

with torch.no_grad():
  for pixel_values, decoder_input_ids, prompt_start_indx, gt_prompt in tqdm(valid_dataloader_sub):
    # print(batch) ## printing all the details of four values in each row i.e pixel value, decoder_input_ids, prompt start token index and gt_prompt
    # print('*************************************************')

    pixel_values = pixel_values.to(device)
    decoder_input_ids = decoder_input_ids.to(device)
    prompt_start_indx = prompt_start_indx.to(device)
    gt_prompt = gt_prompt

    # print(pixel_values.shape, decoder_input_ids.shape, prompt_start_indx.shape)

    # # for decoder_input_id, ans_start_indx in zip(decoder_input_ids, prompt_start_indx):
    # #   print(decoder_input_id, ans_start_indx)

    decoder_input_question_ids = pad_sequence(
            [decoder_input_id[ : ans_start_indx + 1] for decoder_input_id, ans_start_indx in zip(decoder_input_ids, prompt_start_indx)],
            batch_first=True,
        )


    # print(decoder_gt_question_prompt, decoder_gt_question_prompt.shape)
    # e.g tensor([[    0, 57525, 42543, .... 36209, 57526, 57527]]) , torch.Size([1, 20])
    # print('*************************************************')

    outputs_gen = model.generate(pixel_values,
                                 decoder_input_ids = decoder_input_question_ids,
                                 max_length=max_length,
                                 early_stopping=True,
                                 pad_token_id=processor.tokenizer.pad_token_id,
                                 eos_token_id=processor.tokenizer.eos_token_id,
                                 use_cache=True,
                                 num_beams=1,
                                 bad_words_ids=[[processor.tokenizer.unk_token_id]],
                                 return_dict_in_generate=True,
                                 )

    # print(outputs_gen)
    # print(outputs_gen.sequences)
    # generated_sequences = outputs_gen.sequences
    # for sequence in generated_sequences:
    #   generated_text = processor.tokenizer.decode(sequence, skip_special_tokens=True)
    #   print(generated_text)

    # Process and evaluate the predictions and answers
    predictions = []
    for seq in processor.tokenizer.batch_decode(outputs_gen.sequences):
      # print(seq)
      seq = seq.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
      seq = re.sub(r"<.*?>", "", seq, count=1).strip()
      # print(seq)
      predictions.append(seq)
    # print(predictions)

    scores = []
    for pred_prompt, actual_prompt in zip(predictions, gt_prompt):
      # print(pred_prompt)
      # print(actual_prompt)
      pred_prompt = re.sub(r"(?:(?<=>) | (?=</s_))", "", pred_prompt)
      # print(pred_prompt)
      actual_prompt = re.sub(r"<.*?>", "", actual_prompt, count=1)
      actual_prompt = actual_prompt.replace(processor.tokenizer.eos_token, "")
      # print(actual_prompt)
      # print(edit_distance(pred_prompt, actual_prompt))
      scores.append(edit_distance(pred_prompt, actual_prompt) / max(len(pred_prompt), len(actual_prompt)))

      # print(scores)

      if config.get("verbose", False) and len(scores) == 1:
        print(f"Predicted Answer: {pred_prompt}")
        print(f"Actual Answer: {actual_prompt}")
        print(f"Normed Edit Distance Score: {scores[0]}")

    print('*************************************************')



  0%|          | 0/4 [00:00<?, ?it/s]

Predicted Answer: <s_question>What is the ‘actual’ value per 1000, during the year 1975?</s_question><s_answer>1980</s_answer>
Actual Answer: <s_question>What is the ‘actual’ value per 1000, during the year 1975?</s_question><s_answer>0.28</s_answer>
Normed Edit Distance Score: 0.037037037037037035
*************************************************
Predicted Answer: <s_question>What is the name of the company?</s_question><s_answer>itc</s_answer>
Actual Answer: <s_question>What is the name of the company?</s_question><s_answer>ITC Limited</s_answer>
Normed Edit Distance Score: 0.10112359550561797
*************************************************
Predicted Answer: <s_question>To whom is the document sent?</s_question><s_answer>Guillermo Waren</s_answer>
Actual Answer: <s_question>To whom is the document sent?</s_question><s_answer>Paul</s_answer>
Normed Edit Distance Score: 0.15555555555555556
*************************************************
Predicted Answer: <s_question>What is ITC's b

In [ ]:
pixel_values.shape

torch.Size([2, 3, 1280, 960])

In [ ]:
processor.tokenizer.eos_token, processor.tokenizer.pad_token

('</s>', '<pad>')

In [ ]:
processor.tokenizer.eos_token_id, processor.tokenizer.pad_token_id

(2, 1)

In [ ]:
decoder_input_question_ids[0]

tensor([    0, 57529, 42543, 52743, 53821, 22959, 28431, 46192, 54350,  2587,
        55479,     4, 46259, 49224, 25515, 48941, 55662, 14725, 45901, 36209,
        57524, 57526])

In [ ]:
outputs_gen[0]

tensor([[    0, 57529, 42543, 52743, 53821, 22959, 28431, 46192, 54350,  2587,
         55479,     4, 46259, 49224, 25515, 48941, 55662, 14725, 45901, 36209,
         57524, 57526, 40990, 28019, 28431, 46192, 54350, 46192,  2586,  9971,
         23417,  6274, 57245, 48941, 52917, 57523,     2],
        [    0, 57529, 42543, 52743, 48941, 55856,  2587,  9809, 27336, 36209,
         57524, 57526,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0, 48941,  6156,  3105, 57523,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1]])

In [ ]:
processor.tokenizer.decode(outputs_gen[0][0].squeeze())

"<s><s_question> What is ITC's brand of Atta featured in the advertisement?</s_question><s_answer> itc's brands: an asset for the Nation</s_answer></s>"

In [ ]:
## Cosine Similarity Score of answer from whole string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(prediction, actual):
  start_token = "<s_answer>"
  end_token = "</s_answer>"

  # Extract the <s_answer> and </s_answer> substrings
  prediction_answer = prediction[prediction.index(start_token) + len(start_token):prediction.index(end_token)]
  actual_answer = actual[actual.index(start_token) + len(start_token):actual.index(end_token)]

  # Vectorize the extracted answers
  vectorizer = TfidfVectorizer()
  tfidf_matrix = vectorizer.fit_transform([prediction_answer, actual_answer])

  # Calculate cosine similarity
  similarity_score = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
  return similarity_score



In [ ]:
## Validation section using Cosine Similarity
model.eval()

with torch.no_grad():
  for pixel_values, decoder_input_ids, prompt_start_indx, gt_prompt in tqdm(valid_dataloader_sub):
    # print(batch) ## printing all the details of four values in each row i.e pixel value, decoder_input_ids, prompt start token index and gt_prompt
    # print('*************************************************')

    pixel_values = pixel_values.to(device)
    decoder_input_ids = decoder_input_ids.to(device)
    prompt_start_indx = prompt_start_indx.to(device)
    gt_prompt = gt_prompt

    # print(pixel_values.shape, decoder_input_ids.shape, prompt_start_indx.shape)

    # # for decoder_input_id, ans_start_indx in zip(decoder_input_ids, prompt_start_indx):
    # #   print(decoder_input_id, ans_start_indx)

    decoder_input_question_ids = pad_sequence(
            [decoder_input_id[ : ans_start_indx + 1] for decoder_input_id, ans_start_indx in zip(decoder_input_ids, prompt_start_indx)],
            batch_first=True,
        )


    # print(decoder_gt_question_prompt, decoder_gt_question_prompt.shape)
    # e.g tensor([[    0, 57525, 42543, .... 36209, 57526, 57527]]) , torch.Size([1, 20])
    # print('*************************************************')

    outputs_gen = model.generate(pixel_values,
                                 decoder_input_ids = decoder_input_question_ids,
                                 max_length=max_length,
                                 early_stopping=True,
                                 pad_token_id=processor.tokenizer.pad_token_id,
                                 eos_token_id=processor.tokenizer.eos_token_id,
                                 use_cache=True,
                                 num_beams=1,
                                 bad_words_ids=[[processor.tokenizer.unk_token_id]],
                                 return_dict_in_generate=True,
                                 )

    # print(outputs_gen)
    # print(outputs_gen.sequences)
    # generated_sequences = outputs_gen.sequences
    # for sequence in generated_sequences:
    #   generated_text = processor.tokenizer.decode(sequence, skip_special_tokens=True)
    #   print(generated_text)

    # Process and evaluate the predictions and answers
    predictions = []
    for seq in processor.tokenizer.batch_decode(outputs_gen.sequences):
      # print(seq)
      seq = seq.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
      seq = re.sub(r"<.*?>", "", seq, count=1).strip()
      # print(seq)
      predictions.append(seq)
    # print(predictions)

    scores = []
    for pred_prompt, actual_prompt in zip(predictions, gt_prompt):
      # print(pred_prompt)
      # print(actual_prompt)
      pred_prompt = re.sub(r"(?:(?<=>) | (?=</s_))", "", pred_prompt)
      # print(pred_prompt)
      actual_prompt = re.sub(r"<.*?>", "", actual_prompt, count=1)
      actual_prompt = actual_prompt.replace(processor.tokenizer.eos_token, "")
      # print(actual_prompt)

      # model.get_encoder
      # # Calculate cosine similarity between pred_prompt and actual_prompt
      # pred_vector = model.get_encoder(pred_prompt)  # Replace 'model.encode' with the actual encoding function of your model
      # actual_vector = model.get_encoder(actual_prompt)  # Replace 'model.encode' with the actual encoding function of your model
      similarity_score = calculate_cosine_similarity(pred_prompt, actual_prompt)
      scores.append(similarity_score)


      # print(scores)

      if config.get("verbose", False) and len(scores) == 1:
        print(f"Prediction: {pred_prompt}")
        print(f"Actual Answer: {actual_prompt}")
        print(f"Normed Similarity Score: {scores[0]}")

    print('*************************************************')



  0%|          | 0/4 [00:00<?, ?it/s]

Prediction: <s_question>What is the ‘actual’ value per 1000, during the year 1975?</s_question><s_answer>1980</s_answer>
Actual Answer: <s_question>What is the ‘actual’ value per 1000, during the year 1975?</s_question><s_answer>0.28</s_answer>
Normed Similarity Score: 0.0
*************************************************
Prediction: <s_question>What is the name of the company?</s_question><s_answer>itc</s_answer>
Actual Answer: <s_question>What is the name of the company?</s_question><s_answer>itc limited</s_answer>
Normed Similarity Score: 0.5797386715376658
*************************************************
Prediction: <s_question>To whom is the document sent?</s_question><s_answer>Guillermo Waren</s_answer>
Actual Answer: <s_question>To whom is the document sent?</s_question><s_answer>Paul</s_answer>
Normed Similarity Score: 0.0
*************************************************
Prediction: <s_question>What is ITC's brand of Atta featured in the advertisement?</s_question><s_answer>i

## S-5: Save and Reload Processor as well as Model for Test Inference Check
#### ** Note:
- When saving a PyTorch model using torch.save(model, 'model.pth'), only the model's parameters and architecture are saved. The model's configuration or any other related files are not automatically saved along with it.

- If you have a separate configuration file for your model, you need to save it separately. You can follow a similar approach to saving the model's processor and config

```
from transformers import BertConfig, BertModel
# if model is on hugging face Hub
model = BertModel.from_pretrained("bert-base-uncased")
# from local folder
model = BertModel.from_pretrained("./test/saved_model/")

```
- To save the entire tokenizer/processor, you should use save_pretrained()

```
BASE_MODEL = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.save_pretrained("./models/tokenizer/")
tokenizer2 = DistilBertTokenizer.from_pretrained("./models/tokenizer/")


Pro-2
model = MyModel(num_classes).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=1e-2)
output_model = './models/model_xlnet_mid.pth'

save
def save(model, optimizer):
    # save
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, output_model)

save(model, optimizer)

load
checkpoint = torch.load(output_model, map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

```

In [ ]:
# from datasets import load_from_disk
# from transformers import DonutProcessor

## COMMENT IN in case you want to save the processed dataset to disk in case of error later
# processed_dataset.save_to_disk("processed_dataset")
processor.save_pretrained("/content/processor_model_config/processor")

In [ ]:
## COMMENT IN in case you want to load the processed dataset from disk in case of error later
# processed_dataset = load_from_disk("processed_dataset")
loaded_processor = DonutProcessor.from_pretrained("/content/processor_model_config/processor")

In [ ]:
from pathlib import Path
# pathlib is a Python module for working with file paths and directories
# Path is a class in the pathlib module that represents a path to a file or directory

In [ ]:
## Saving our PyTorch Model

# 1. Create models directory
model_path = Path('/content/processor_model_config/Donut-Model-Sub-INC')   # creates a Path object named model_path representing a directory named "Models"
model_path.mkdir(parents=True, exist_ok = True)   # mkdir() method is called on model_path to create the directory if it does not already exist
# parents=True argument creates any necessary parent directories that do not exist yet
# exist_ok=True argument makes the method not raise an error if the directory already exists

# 2. Crerate model save path
model_Name = '01_model_sub.pth'
model_save_path = model_path/model_Name   # new Path object is created

# 3. Save the model state_dict
print(f'Saving model to: {model_save_path}')
torch.save(obj = model.state_dict(),
           f = model_save_path)
# obj: The object that needs to be saved.
# f: The file name or file object or path like object where the serialized object will be stored.
# Here, model_save_path is passed as the file name where the serialized object will be stored.

Saving model to: /content/processor_model_config/Donut-Model-Sub-INC/01_model_sub.pth


In [ ]:
## Before Loading the model checking the parameter
# model.state_dict()

##### Imp info regarding save and load model:
- https://wandb.ai/wandb/common-ml-errors/reports/How-to-Save-and-Load-Models-in-PyTorch--VmlldzozMjg0MTE

In [ ]:
import pickle

In [ ]:
model_cofig = model.config

In [ ]:
# Save the model config using pickle
with open('/content/processor_model_config/model_cofig.pkl', 'wb') as f:
  pickle.dump(model_cofig, f)

In [ ]:
# Load the config using pickle
with open('/content/processor_model_config/model_cofig.pkl', 'rb') as f:
  loaded_config = pickle.load(f)

In [ ]:
# Load the Model
# Create a new instance of the model with the loaded config
loaded_model = VisionEncoderDecoderModel.from_pretrained(model_checkpoint, config = loaded_config)

In [ ]:
# Load the saved model's state_dict
loaded_model.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

#### Imp Note on model.eval():

- When you load the model for inference using loaded_model_1 = VisionEncoderDecoderModel.from_pretrained(model_checkpoint, config=config), the model retains its state, including the evaluation mode set during training and validation. Therefore, there is no need to call model.eval() again after loading the model.

- You can directly proceed with the inference using loaded_model_1 without calling model.eval() in this case.

In [ ]:
from IPython.display import display

In [ ]:
# x = hf_test_sub_ocr['full_path_image'][5]
# y = Image.open(x).convert("RGB")
# display(y)

## S-6.1: Inference check from Test Dataset

In [ ]:
## 1. test prediction on DOCVQA test dataset where question is fixed

device = "cuda" if torch.cuda.is_available() else "cpu"
loaded_model.to(device)

VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0-1): 2 x DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
           

In [ ]:

## Test Inference custom function for DOCVQA
def run_test_prediction(hf_test_samples, model=loaded_model, processor=loaded_processor):

  k = int(input("Enter the row sample number: "))

  ## Reading Image
  test_image_path = hf_test_samples['full_path_image'][k]
  test_image = Image.open(test_image_path).convert("RGB")
  # display(test_image)

  ## Printing Pixel Value shape
  test_pixel_values = processor(test_image, return_tensors="pt").pixel_values
  print(test_pixel_values.shape)


  ## Creating test task prompt
  test_task_prompt = "<s><s_question>{user_input}</s_question><s_answer>"

  test_question = hf_test_samples['question'][k]                        ## "When is the coffee break?"

  test_prompt = test_task_prompt.replace("{user_input}", test_question)
  print(test_prompt)

  test_decoder_input_ids = processor.tokenizer(test_prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]
  # print(test_decoder_input_ids)

  test_outputs = model.generate(test_pixel_values.to(device),
                               decoder_input_ids = test_decoder_input_ids.to(device),
                               max_length = model.decoder.config.max_position_embeddings,
                               early_stopping = True,
                               pad_token_id = processor.tokenizer.pad_token_id,
                               eos_token_id = processor.tokenizer.eos_token_id,
                               use_cache = True,
                               num_beams = 1,
                               bad_words_ids = [[processor.tokenizer.unk_token_id]],
                               return_dict_in_generate = True,
                               output_scores = True)
  # print(test_outputs)
  test_seq_prediction = processor.batch_decode(test_outputs.sequences)[0]
  test_seq_prediction = test_seq_prediction.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
  test_seq_prediction = re.sub(r"<.*?>", "", test_seq_prediction, count=1).strip()  # remove first task start token
  print(test_seq_prediction)

  test_seq_prediction_json = processor.token2json(test_seq_prediction)

  return test_seq_prediction_json



In [ ]:
test_json_result = run_test_prediction(hf_test_sub_ocr)
test_json_result

Enter the row sample number: 5
torch.Size([1, 3, 1280, 960])
<s><s_question>What is the table number?</s_question><s_answer>
<s_question> What is the table number?</s_question><s_answer> 11</s_answer>


{'question': 'What is the table number?', 'answer': '11'}

## S-6.2: Inference check from User Input

In [ ]:
## 2. test prediction from any image along with any question asked by the user input

device = "cuda" if torch.cuda.is_available() else "cpu"
loaded_model.to(device)

## Test Inference custom function for DOCVQA
def run_prediction(user_test_sample, model=loaded_model, processor=loaded_processor):

  user_test_image = Image.open(user_test_sample).convert("RGB")
  display(user_test_image)

  pixel_values = processor(user_test_image, return_tensors="pt").pixel_values
  print(pixel_values.shape)

  user_task_prompt = "<s><s_question>{user_input}</s_question><s_answer>"
  user_question =   str(input("Write your Question: "))                       ## "When is the coffee break?"
  prompt = user_task_prompt.replace("{user_input}", user_question)
  decoder_input_ids = processor.tokenizer(prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]

  test_outputs = model.generate(pixel_values.to(device),
                               decoder_input_ids = decoder_input_ids.to(device),
                               max_length = model.decoder.config.max_position_embeddings,
                               early_stopping = True,
                               pad_token_id = processor.tokenizer.pad_token_id,
                               eos_token_id = processor.tokenizer.eos_token_id,
                               use_cache = True,
                               num_beams = 1,
                               bad_words_ids = [[processor.tokenizer.unk_token_id]],
                               return_dict_in_generate = True,
                               output_scores = True)

  test_seq_prediction = processor.batch_decode(test_outputs.sequences)[0]
  test_seq_prediction = test_seq_prediction.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
  test_seq_prediction = re.sub(r"<.*?>", "", test_seq_prediction, count=1).strip()  # remove first task start token
  print(test_seq_prediction)

  test_seq_prediction_json = processor.token2json(test_seq_prediction)

  return test_seq_prediction_json



In [ ]:
## User image path
test_img = ''

In [ ]:
user_json_result = run_prediction(test_img)
user_json_result

In [ ]:
# ## Finding Training Accuracy

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Move the model to the device
# model = model.to(device)


# # Set the optimizer
# optimizer = Adam(model.parameters(), lr=0.0001)

# # Set the number of training epochs
# num_epochs = 3

# for epoch in range(num_epochs):

#   # Training loop
#   model.train()

#   print(f"Epoch {epoch + 1}/{num_epochs}")

#   total_loss = 0.0    # Total loss across all batches in an epoch
#   total_correct = 0   # Total correct across all batches in an epoch
#   total_samples = 0   # Effective when batch_size is more than 1
#   total_samples_mask_level = 0 # Total number of non-padding and ignored tokens

#   # Iterate over the training dataset
#   for pixel_values, decoder_input_ids, labels in tqdm(train_dataloader_sub):

#     pixel_values = pixel_values.to(device)
#     decoder_input_ids = decoder_input_ids.to(device)
#     labels = labels.to(device)

#     ## Finding outputs prediction
#     outputs = model(pixel_values=pixel_values, decoder_input_ids=decoder_input_ids[:, :-1], labels=labels[:, 1:])

#     ## Compute  the Loss
#     loss = outputs.loss
#     total_loss += loss.item()

#     # Backward pass and optimization
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()


#     # Compute the Accuracy (3-Ways)
#     ## Pro-2
#     total_samples += labels.size(0)

#     ## Pro-3 Mask level
#     # Create a mask to exclude padding tokens & ignored tokens from accuracy calculation
#     mask = (labels != -100)
#     # Update the total number of non-padding tokens
#     total_samples_mask_level += mask.sum().item()

#     # Finding logits value
#     logits = outputs.logits
#     predictions = logits.argmax(dim=-1) # index value of maximum score for each token
#     correct = (predictions == labels).sum().item()    ## Number of correct in each row
#     total_correct += correct    # Cumulative correct then lastly total correct prediction from each epoch


#   avg_epoch_loss_batch_level = total_loss / len(train_dataloader_sub)   # average loss per batch i.e. normalize the loss value

#   avg_epoch_accuracy_batch_level = total_correct / len(train_dataloader_sub) # average per epoch accuracy at batch level

#   avg_epoch_accuracy_sample_level = total_correct / total_samples # average per epoch accuracy at based on number of data point

#   avg_epoch_accuracy_mask_level = (total_correct / total_samples_mask_level) * 100 # average per epoch accuracy at based on mask or non padding tokens

#   # If batch_size = 1 , so we can use anything len(train_dataloader_sub) or  len(train_dataset)

#   print(f"Training Loss Batch Level: {avg_epoch_loss_batch_level:.4f} :::: Training Accuracy Batch Level: {avg_epoch_accuracy_batch_level:.3f}%")
#   print(f"Training Accuracy Entire Sample Level: {avg_epoch_accuracy_sample_level:.3f}% :::: Training Accuracy Mask Level: {avg_epoch_accuracy_mask_level:.3f}%")
#   print('*****************************************************************************************************************************')


#   ## Validation section
#   model.eval()

#   with torch.no_grad():
#     for pixel_values, decoder_input_ids, prompt_start_indx, gt_prompt in tqdm(valid_dataloader_sub):
#       # print(batch) ## printing all the details of four values in each row i.e pixel value, decoder_input_ids, prompt start token index and gt_prompt
#       # print('*************************************************')

#       pixel_values = pixel_values.to(device)
#       decoder_input_ids = decoder_input_ids.to(device)
#       prompt_start_indx = prompt_start_indx.to(device)
#       gt_prompt = gt_prompt
#       # print(pixel_values.shape, decoder_input_ids.shape, prompt_start_indx.shape)

#       # # for decoder_input_id, ans_start_indx in zip(decoder_input_ids, prompt_start_indx):
#       # #   print(decoder_input_id, ans_start_indx)

#       decoder_input_question_ids = pad_sequence(
#               [decoder_input_id[ : ans_start_indx + 1] for decoder_input_id, ans_start_indx in zip(decoder_input_ids, prompt_start_indx)],
#               batch_first=True,
#           )

#       # print(decoder_gt_question_prompt, decoder_gt_question_prompt.shape)
#       # e.g tensor([[    0, 57525, 42543, .... 36209, 57526, 57527]]) , torch.Size([1, 20])
#       # print('*************************************************')

#       outputs_gen = model.generate(pixel_values,
#                                   decoder_input_ids = decoder_input_question_ids,
#                                   max_length=max_length,
#                                   early_stopping=True,
#                                   pad_token_id=processor.tokenizer.pad_token_id,
#                                   eos_token_id=processor.tokenizer.eos_token_id,
#                                   use_cache=True,
#                                   num_beams=1,
#                                   bad_words_ids=[[processor.tokenizer.unk_token_id]],
#                                   return_dict_in_generate=True,
#                                   )

#       # print(outputs_gen)
#       # print(outputs_gen.sequences)
#       # generated_sequences = outputs_gen.sequences
#       # for sequence in generated_sequences:
#       #   generated_text = processor.tokenizer.decode(sequence, skip_special_tokens=True)
#       #   print(generated_text)

#       # Process and evaluate the predictions and answers
#       predictions = []
#       for seq in processor.tokenizer.batch_decode(outputs_gen.sequences):
#         # print(seq)
#         seq = seq.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
#         seq = re.sub(r"<.*?>", "", seq, count=1).strip()
#         # print(seq)
#         predictions.append(seq)
#       # print(predictions)

#       scores = []
#       for pred_prompt, actual_prompt in zip(predictions, gt_prompt):
#         # print(pred)
#         # print(prom)
#         pred_prompt = re.sub(r"(?:(?<=>) | (?=</s_))", "", pred_prompt)
#         actual_prompt = re.sub(r"<.*?>", "", actual_prompt, count=1)
#         actual_prompt = actual_prompt.replace(processor.tokenizer.eos_token, "")
#         scores.append(edit_distance(pred_prompt, actual_prompt) / max(len(pred_prompt), len(actual_prompt)))

#         if config.get("verbose", False) and len(scores) == 1:
#           print(f"Prediction: {pred_prompt}")
#           print(f"Actual Answer: {actual_prompt}")
#           print(f"Normed ED: {scores[0]}")

#       print('*************************************************')


